In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import matplotlib.dates as md
import seaborn as sns
import datetime as datetime
import numpy as np
import os

%matplotlib inline

plt.style.use('default')

sns.set(style="whitegrid")

import warnings 
warnings.filterwarnings('ignore')

my_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

Limpieza_Clicks = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Clicks.ipynb'))
Limpieza_Auctions = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Auctions.ipynb'))
Limpieza_Installs = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Installs.ipynb'))
Limpieza_Events = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Events.ipynb'))

In [3]:
%run $Limpieza_Clicks
%run $Limpieza_Auctions
%run $Limpieza_Installs
%run $Limpieza_Events

In [4]:
training_set = auctions
training_set.sort_values(ascending=True,by='date',inplace=True)

In [5]:
#Separamos el set de entrenamiento en 5 "ventanas" de 3 días cada una.
ventana1 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-18')) & (training_set['date'] < pd.to_datetime('2019-04-21'))]
ventana2 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-19')) & (training_set['date'] < pd.to_datetime('2019-04-22'))]
ventana3 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-20')) & (training_set['date'] < pd.to_datetime('2019-04-23'))]
ventana4 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-21')) & (training_set['date'] < pd.to_datetime('2019-04-24'))]
ventana5 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-22')) & (training_set['date'] < pd.to_datetime('2019-04-25'))]

In [46]:
ventana1['rg_min'] = ventana1.groupby('device_id')['date'].transform(min)
ventana1['rg_max'] = ventana1.groupby('device_id')['date'].transform(max)

train_st = ventana1[['device_id','rg_min','rg_max']].drop_duplicates()
train_st.columns = ['device_id','v1_min','v1_max']

In [47]:
ventana2['rg_min'] = ventana2.groupby('device_id')['date'].transform(min)
ventana2['rg_max'] = ventana2.groupby('device_id')['date'].transform(max)

train_st = pd.merge(train_st,ventana2[['device_id','rg_min','rg_max']].drop_duplicates(),on='device_id',how='inner')
train_st.columns = ['device_id','v1_min','v1_max','v2_min','v2_max']

In [48]:
ventana3['rg_min'] = ventana3.groupby('device_id')['date'].transform(min)
ventana3['rg_max'] = ventana3.groupby('device_id')['date'].transform(max)

train_st = pd.merge(train_st,ventana3[['device_id','rg_min','rg_max']].drop_duplicates(),on='device_id',how='inner')
train_st.columns = ['device_id','v1_min','v1_max','v2_min','v2_max','v3_min','v3_max']

In [49]:
ventana4['rg_min'] = ventana4.groupby('device_id')['date'].transform(min)
ventana4['rg_max'] = ventana4.groupby('device_id')['date'].transform(max)

train_st = pd.merge(train_st,ventana4[['device_id','rg_min','rg_max']].drop_duplicates(),on='device_id',how='inner')
train_st.columns = ['device_id','v1_min','v1_max','v2_min','v2_max','v3_min','v3_max','v4_min','v4_max']

In [50]:
ventana5['rg_min'] = ventana5.groupby('device_id')['date'].transform(min)
ventana5['rg_max'] = ventana5.groupby('device_id')['date'].transform(max)

train_st = pd.merge(train_st,ventana5[['device_id','rg_min','rg_max']].drop_duplicates(),on='device_id',how='inner')
train_st.columns = ['device_id','v1_min','v1_max','v2_min','v2_max','v3_min','v3_max','v4_min','v4_max','v5_min','v5_max']

In [51]:
train_st['st_v1'] =  abs((train_st['v1_max'] - train_st['v2_min']).dt.total_seconds())
train_st['st_v2'] =  abs((train_st['v2_max'] - train_st['v3_min']).dt.total_seconds())
train_st['st_v3'] =  abs((train_st['v3_max'] - train_st['v4_min']).dt.total_seconds())
train_st['st_v4'] =  abs((train_st['v4_max'] - train_st['v5_min']).dt.total_seconds())

In [52]:
train_st = train_st[['device_id','st_v1','st_v2','st_v3','st_v4']]

In [53]:
train_st.head()

,device_id,st_v1,st_v2,st_v3,st_v4
0,1826643666390887030,138678.511676,110829.657687,24.046311,27.711484
1,7037174172278258682,163004.973014,167418.683131,168594.712797,163709.030912
2,1228982273563226229,160231.705578,141579.669077,169169.880384,62493.633896
3,2544156497510885464,170579.934232,172179.012691,170581.438113,172556.621229
4,8160556786289359332,113155.447099,101359.822653,88628.690546,5771.857343


# FEATURES

In [89]:
events_v1 = events.loc[(events['date'] > pd.to_datetime('2019-04-18')) & (events['date'] < pd.to_datetime('2019-04-21'))]
df_features = events_v1.groupby('ref_hash').agg({'ref_hash':'size'})

In [90]:
df_features.columns = ['cantidad_de_eventos']

In [91]:
Week_Day = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]

for dia in Week_Day:
    eventos_por_dia = events[events['date'].dt.day_name() == dia].groupby('ref_hash').agg({'ref_hash':'size'})
    eventos_por_dia.columns = ['cantidad_eventos_{}'.format(dia)]
    
    df_features = df_features.merge(eventos_por_dia, how='outer', left_index=True, right_index=True)
    
df_features = df_features.fillna(0)

In [92]:
df_features.reset_index(inplace=True)

In [93]:
df_features = pd.merge(train_st[['device_id']],df_features,left_on='device_id',right_on='ref_hash',how='inner')

In [94]:
df_features.head()

,device_id,ref_hash,cantidad_de_eventos,cantidad_eventos_Monday,cantidad_eventos_Tuesday,cantidad_eventos_Wednesday,cantidad_eventos_Thursday,cantidad_eventos_Friday,cantidad_eventos_Saturday,cantidad_eventos_Sunday
0,7037174172278258682,7037174172278258682,0.0,0.0,0.0,0.0,0.0,65.0,0.0,0.0
1,2844535553398493162,2844535553398493162,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,1680272311925142702,1680272311925142702,5.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0
3,3927392842794041835,3927392842794041835,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
4,2227766999717522159,2227766999717522159,5.0,0.0,1.0,0.0,2.0,3.0,0.0,0.0


In [95]:
df_features.shape

(88652, 10)

In [96]:
del df_features['device_id']

In [97]:
df_features.sort_values(by='ref_hash',inplace=True)

In [98]:
label1 = (train_st.loc[train_st['device_id'].isin(df_features['ref_hash'])])[['device_id','st_v1']]

In [99]:
label1.shape

(88652, 2)

In [100]:
label1['st_v1'].replace(
    to_replace=[0],
    value=259200,
    inplace=True
)

label1['binary'] = np.where(label1['st_v1']>=259200,False,True)
label1.sort_values(by='device_id',inplace=True)

In [101]:
del label1['device_id']
label1 = label1[['binary','st_v1']].to_records(index=False) 

# Algoritmo

In [102]:
from sksurv.linear_model import CoxPHSurvivalAnalysis

estimator = CoxPHSurvivalAnalysis()
estimator.fit(df_features,label1)

CoxPHSurvivalAnalysis(alpha=0, n_iter=100, tol=1e-09, verbose=0)

In [103]:
pd.Series(estimator.coef_, index=df_features.columns)

ref_hash                     -8.357441e-22
cantidad_de_eventos           1.150665e-05
cantidad_eventos_Monday      -3.380476e-04
cantidad_eventos_Tuesday      4.059600e-04
cantidad_eventos_Wednesday   -5.670885e-04
cantidad_eventos_Thursday     2.586518e-04
cantidad_eventos_Friday       6.406885e-05
cantidad_eventos_Saturday     3.038192e-04
cantidad_eventos_Sunday       1.380709e-04
dtype: float64

#### El resultado es un vector de coeficientes, uno para cada variable, donde cada valor corresponde a la relación del registro y evento

In [104]:
prediction = estimator.predict(df_features)

In [105]:
estimator.score(df_features, label1)

0.5021970863822909

# KNN

In [107]:
set_training = pd.merge(train_st[['device_id','st_v1']],df_features, how='inner', left_on='device_id',right_on='ref_hash')

In [108]:
del set_training['device_id']

In [109]:
columns = list(set_training.columns)
columns.remove('st_v1')

In [110]:
from sklearn.model_selection import train_test_split

#TRAIN_TEST_SPLIT
#Split arrays or matrices into random train and test subsets
X_train, X_test, Y_train, Y_test = train_test_split(set_training.loc[:, columns],
                                                    set_training['st_v1'],
                                                    test_size=0.2,
                                                    random_state=43)

In [111]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics

k=5 #-->Hiperparametro
neigh = KNeighborsRegressor(n_neighbors = k ,    #Cantidad de vecinos a usar --> K hiperparametro.
                            n_jobs=-1,           #Define la cantidad de procesadores a usar, -1 todos los disponibles.
                            weights='distance',  #Se define el peso como la inversa a la distancia, los mas cercanos tienen mayor influencia.
                            algorithm='brute'    #Fuerza bruta para el algorimo, podría ser KDTree o sino automatico.
                           )
#En weights se puede pasar una funcion definida por el usuario--> Segundo hiperparametro posible de KNN.

In [112]:
neigh.fit(X_train,Y_train) #Fit the model using X as training data and Y as target values

KNeighborsRegressor(algorithm='brute', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
          weights='distance')

In [113]:
predicted = neigh.predict(X_test) #Predict the target for the provided data

In [114]:
neigh.score(X_test, Y_test)

-0.3960931039558926